In [5]:
import pandas as pd
from pathlib import Path
import sqlalchemy as sa
from dotenv import load_dotenv
import os
import psutil
import regex as re
import numpy as np
from tqdm import tqdm
import datetime
from prepay_utilities import create_amortization_lookup, show_slice, amortize
load_dotenv()

# set options for pandas and numpy formatting (no errors should occur if these are ommitted)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None  # default='warn'
np.set_printoptions(formatter={'float':"{:6.5g}".format})

engine = sa.create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
			.format(host=os.getenv('host'), db=os.getenv('db'), user=os.getenv('uname'), pw=os.getenv('password')))

### Loan class definition

In [5]:
class Loan:
    ObservationNmb: int
    maturity_dt: datetime.date
    origination_dt: datetime.date
    lender_status: int
    all_pmts: np.ndarray
    
    def __init__(self, nmb, m_dt, o_dt, l_st,pmts):
        self.ObservationNmb = nmb
        self.maturity_dt = m_dt
        self.origination_dt = o_dt
        self.all_pmts = pmts
        self.lender_status = l_st
        
    def find_payoff_date(self):
        pass
 
 

### Loading from .pickle files

In [3]:
import pickle
# with open('loans_new.pickle', 'rb') as f:
#     loans = pickle.load(f)
# with open('prepay_data.pickle', 'wb') as f:
#     pickle.dump(x_1, f)
# import pickle
with open('pickle_files/prepay_data.pickle', 'rb') as f:
   x_1= pickle.load(f)



Build Dataframe 
* Format dates
* Drop duplicates and nan

In [6]:
def build_df_list(prepayment_arr, cols):
    df_list = []
    for d in prepayment_arr:
        df_list.append(pd.DataFrame(data=d, columns= cols))
    return df_list

columns = ['ObservationNmb', 'Origination_dt', 'MaturityDt', 'EffectiveDt', 'GeneralLedgerCd', 'TransactionAmt', 'TransactionBalanceAmt']
df_list = build_df_list(x_1, columns)
df = pd.concat(df_list)
df['EffectiveDt'] = pd.to_datetime(df['EffectiveDt'])
df['Origination_dt'] = pd.to_datetime(df['Origination_dt'])
df['MaturityDt'] = pd.to_datetime(df['MaturityDt'])
df = df[df['EffectiveDt'].dt.year != 1960]
df['ObservationNmb'] = df['ObservationNmb'].astype(str)
df = df.drop_duplicates()
df = df.dropna()
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)



In [7]:
df[(df.TransactionAmt<=1) & (df.TransactionAmt>=-1)]

,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt
1085,1478062,2000-06-22,2025-09-30,2001-07-31,6031,-0.02,415128.00
1166,1478270,2000-01-07,2025-01-31,2002-05-31,6031,0.03,2409760.00
1420,1480771,2000-01-03,2009-04-29,2002-02-28,6031,0.28,138495.00
1427,1480771,2000-01-03,2009-04-29,2002-12-31,6031,0.01,122885.00
1513,1480781,2000-01-03,2007-11-15,2005-11-30,6031,-0.02,260723.00
...,...,...,...,...,...,...,...
407439,2016519,2000-12-28,2011-02-28,2010-10-31,6031,0.22,52596.00
407465,2016520,2000-12-29,2011-07-31,2003-05-31,6031,0.01,60577.30
408159,2016573,2000-12-29,2021-08-31,2001-07-31,6031,0.01,899363.00
408311,2016573,2000-12-29,2021-08-31,2014-03-31,6031,-0.01,422368.00


## Make amortization schedule

In [8]:
print('creating amortization look_up table...')
amortization_lookup = create_amortization_lookup(engine_obj=engine)
amortization_lookup = amortization_lookup.set_index('ObservationNmb')
amortization_lookup= amortization_lookup.fillna(0)
print('done.\n')


creating amortization look_up table...
done.



In [9]:
print('looping through loans...')

sample_dict = {}
for df_x in tqdm(df_list):
    if df_x.any().sum()>0:
        df_x = df_x.drop_duplicates().reset_index().drop(columns='index')
        num = df_x['ObservationNmb'][0]
        lf_dt = df_x['Origination_dt'][0]
        m_dt = df_x['MaturityDt'][0]
        interest_rate = amortization_lookup.loc[num]['BankInterestPct']
        if amortization_lookup.loc[num]['GrossGtyDisbursementAmt']==0:
            loan_amt = df_x['TransactionBalanceAmt'].max()
        else: 
            loan_amt = amortization_lookup.loc[num]['GrossGtyDisbursementAmt']
        try:
            months = int( pd.Timedelta(m_dt - lf_dt)/np.timedelta64(1, 'M'))
        except TypeError:
            months = amortization_lookup.loc[num]['MaturityMthsQty']

        sample_dict[num] = [df_x, pd.DataFrame(amortize(interest_rate, loan_amt, months))]


looping through loans...


  0%|          | 0/9318 [00:00<?, ?it/s]c:\Users\Laramie\anaconda3\envs\dev\lib\site-packages\numpy_financial\_financial.py:226: RuntimeWarning: invalid value encountered in double_scalars
  return -(fv + pv*temp) / fact
 27%|██▋       | 2554/9318 [00:18<00:42, 159.46it/s]c:\Users\Laramie\anaconda3\envs\dev\lib\site-packages\numpy_financial\_financial.py:226: RuntimeWarning: divide by zero encountered in double_scalars
  return -(fv + pv*temp) / fact
100%|██████████| 9318/9318 [01:03<00:00, 147.09it/s]


In [119]:
def return_sample(obs: str, s_dict: dict, _display: bool = False):
    df_actual = s_dict[obs][0][['EffectiveDt','ObservationNmb', 'TransactionAmt', 'TransactionBalanceAmt']]
    df_actual.rename(columns={'TransactionAmt': 'Actual Principal', 
                    'TransactionBalanceAmt': 'Actual Balance'}, inplace=True)
    df_actual.set_index(pd.RangeIndex(df_actual.shape[0]), inplace=True)

    df_scheduled = s_dict[obs][1][['Scheduled Principal','Begging Loan Balance','Ending Loan Balance']]
    df_scheduled.rename(columns={'Begging Loan Balance':'EstimateBegBal',
                        'Ending Loan Balance': 'EstimateEndBal'}, inplace=True)
    df_scheduled = df_scheduled.shift(1)
    df_scheduled.set_index(pd.RangeIndex(df_scheduled.shape[0]), inplace=True)
    
    merged = df_scheduled.merge(df_actual, left_index=True, right_index=True, how='outer')
    merged['Difference'] = 0
    merged.loc[(merged['Actual Principal']<0), 'Difference'] = (merged['Scheduled Principal']+merged['Actual Principal'])
    merged = merged[[ 'EffectiveDt', 'ObservationNmb', 'Scheduled Principal', 'Actual Principal', 'Difference', 'Actual Balance' ,'EstimateBegBal']]
    
    # Set PaidOff to first instance of payoff
    try:
        merged['PaidOff'] = 0
        idx = merged.loc[merged['Actual Balance']==0].iloc[0].name
        merged.at[idx, 'PaidOff'] = 1
    except IndexError:
        pass
    
    # Chop length of dataframe by getting rid of unnessecary rows (payment string has ended but amortization continues)
    chop_here = (merged[merged.ObservationNmb.isnull()].iloc[0].name)
    merged = merged.iloc[0:chop_here]
    
    merged['Actual Balance delta'] = merged['Actual Balance'].diff()
    # merged = merged.tail(merged.shape[0]-1)
    # po_idx = ((merged[merged.ObservationNmb.isnull()].iloc[0].name)-2)
    if _display:
        print('estimated paid off row:')
        # display(merged.iloc[[po_idx]])
        display(merged.head(60))
        return None
    else:
        return merged

In [1]:
import random
obs = list(sample_dict.keys())
# random.choice(obs)
# '1483774'
samp = return_sample('2011851', sample_dict, False)

def show_around_payoff():
    here = samp[samp.PaidOff==1].index[0]
    display(samp.iloc[here-5:(here+5)])
    
round((samp[samp.PaidOff==1].index[0] / samp.index[-1]),2)


NameError: name 'sample_dict' is not defined

In [163]:
samp

,EffectiveDt,ObservationNmb,Scheduled Principal,Actual Principal,Difference,Actual Balance,EstimateBegBal,PaidOff,Actual Balance delta
0,2000-12-31,2011851,NaN,140000.00,0,140000.00,NaN,0,NaN
1,2001-02-28,2011851,1605.00,-3415.37,-1810.37,136585.00,200000.00,0,-3415.00
2,2001-03-31,2011851,1618.00,-1913.72,-295.72,134671.00,198395.00,0,-1914.00
3,2001-04-30,2011851,1631.00,-1828.45,-197.45,132842.00,196777.00,0,-1829.00
4,2001-07-31,2011851,1645.00,48678.70,0,181521.00,195146.00,0,48679.00
5,2001-10-31,2011851,1659.00,-60441.70,-58782.70,121079.00,193501.00,0,-60442.00
6,2001-11-30,2011851,1672.00,-2049.68,-377.68,119030.00,191842.00,0,-2049.00
7,2001-12-31,2011851,1686.00,-2110.64,-424.64,116919.00,190170.00,0,-2111.00
8,2002-02-28,2011851,1700.00,-1151.54,548.46,115768.00,188484.00,0,-1151.00
9,2002-03-31,2011851,1714.00,-2127.10,-413.10,113641.00,186784.00,0,-2127.00


In [109]:
sub_sample = []
# for o in tqdm(obs):
#     samp = return_sample(o, sample_dict, False)
#     first_bal = samp.loc[:,'Actual Balance'].iloc[0]
#     if first_bal <=0:
#         sub_sample.append(samp)
for o in tqdm(obs):
    samp = return_sample(o, sample_dict, False)
    inc_balance_count = samp['Actual Balance delta'][samp['Actual Balance delta']>0].count()
    if inc_balance_count > 2:
        sub_sample.append(samp)

100%|██████████| 7992/7992 [00:30<00:00, 265.75it/s]


In [139]:
inspect = sub_sample[15]

inspect.Difference.values

array([0, 0, -150.66000000000008, -1722.8400000000001, 0, 301.27,
       -16571.5, 328.9, -46.51999999999998, 194.29999999999995,
       -25.230000000000018, 45.84999999999991, 42.460000000000036,
       -1535.4, -4.900000000000091, -1550.25, -34.930000000000064,
       -219.1099999999999, -259.6500000000001, -191.33999999999992,
       -1829.2199999999998, -195.98000000000002, -1882.5,
       -31.180000000000064, -66.69000000000005, -1657.4299999999998,
       -86.59999999999991, -151.83999999999992, -97.24000000000001,
       36.58999999999992, -198.83999999999992, -115.04999999999995,
       -51.66000000000008, -118.76999999999998, -79.97000000000003,
       -19.88000000000011, -81.16000000000008, 17.049999999999955,
       -68.41000000000008, -61.33999999999992, -1756.4499999999998,
       -10.849999999999909, -27.50999999999999, 13.63000000000011,
       -30001.9, -23883.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Build Months from origination and maturity. Trim and cleanse data

In [275]:

df['MnthsFrom_Origination'] = (df['EffectiveDt']-df['Origination_dt'])/np.timedelta64(1,'M')
df['MnthsFrom_Origination'] = df['MnthsFrom_Origination'].astype(float).round(1)
    
df['MnthsFrom_Maturity'] = (df['MaturityDt']-df['EffectiveDt'])/np.timedelta64(1,'M')
df['MnthsFrom_Maturity'] = df['MnthsFrom_Maturity'].astype(int)

# Define PaidOff --> This probably isn't the best way. I think I should start at the end and work backword(which also doesn't sound great but carries more certainty)
df['PaidOff'] = 0
df.loc[ (df['TransactionAmt']<0) & (df['TransactionBalanceAmt']<1000), 'PaidOff' ] = 1

# Adjust subset of loans that record negative balances at the beginning of their term
df.loc[(df['MnthsFrom_Origination']<2) & (df['TransactionBalanceAmt']<0), 'PaidOff'] = 0



In [276]:

# Cleaning the data set:
# 1) Remove transactions that record matching negative balances. These loans have payment strings whose effectivedates are likely in the 
# wrong order
scuffed_data = df[(df['TransactionBalanceAmt'].values==df['TransactionAmt'].values) \
                & (df['TransactionAmt']<0)].index.to_list()
print('cleaning...\n')
df= df.drop(scuffed_data, axis=0)


cleaning...



In [277]:
# 2) Remove loans who's first payment is over 6 months from the OriginationDt. These loans either should have been recorded as 2001 vintage or 
# the payment strings have incorrect effictive dates
# 2a) sort by origination, effectivedt, observationnmb, then the TransactionBalanceAmt
missed_funding_window = df.sort_values(by = ['Origination_dt', 'EffectiveDt', 'ObservationNmb', 'TransactionBalanceAmt'] )
# 2b) groupby observation number, taking the first transaction for each loan
# This SETS THE INDEX TO OBSERVATION NMB
missed_funding_window = missed_funding_window.groupby('ObservationNmb').first()
# 2c) set the window to only those loans whose first transaction occured in 2001
missed_funding_window = missed_funding_window[missed_funding_window['EffectiveDt'].dt.year == 2001]
# 2d) create the test column that we will use to test if a loan is outside its funding window
# 2e) create list of indexes outside the funding window -- do not include previously grabbed indexes
more_s_data = missed_funding_window[missed_funding_window['MnthsFrom_Origination'] > 6][['TransactionBalanceAmt']].index.to_list()
df = df.set_index('ObservationNmb')
print('cleaning...\n')
df = df.drop(more_s_data, axis=0).reset_index()



cleaning...



In [278]:
# 3) Drop out rows with TransactionAmounts of 1 dollar or less. I have not been able to find a discernible reason I would want those records in a payment string
# Usually the result of accounting errors or adhoc principal adjustments we don't need.
scuffed_data = df[(df['TransactionAmt']<=1)&(df['TransactionAmt']>=-1)].index.to_list()
print('cleaning...\n')
df= df.drop(scuffed_data, axis=0)


cleaning...



In [279]:
# 4) Take the earliest payoff date if duplicate payoff dates exist. There will still be multiple payoff dates, but this gets rid of roughly 700 duplicate
# entries that have the same effect with different codes 
df =df.drop_duplicates(subset=['ObservationNmb', 'PaidOff', 'TransactionAmt', 'TransactionBalanceAmt'], keep='first')

print("cleaning...")

cleaning...


In [280]:

# Set PrepayAmt  
df['PrepayAmt'] = 0
df.loc[ (df['PaidOff']==1) , 'PrepayAmt' ] = df['TransactionAmt']

print("------------------------------------------------------------------------------")
print(f"There are {df.shape[0]} transactions in the dataset \n")
print(f"There are {len(df['ObservationNmb'].unique())} loans in the dataset \n")
print("------------------------------------------------------------------------------")

------------------------------------------------------------------------------
There are 379348 transactions in the dataset 

There are 7439 loans in the dataset 

------------------------------------------------------------------------------


408914

In [228]:
df['delta'] = df.groupby('ObservationNmb')['TransactionBalanceAmt'].diff()
df['double_entry'] = df.groupby(['ObservationNmb'])['MnthsFrom_Origination'].diff()


,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt,MnthsFrom_Origination,MnthsFrom_Maturity,PaidOff,PrepayAmt,delta,double_entry
63,1469639,2000-02-01,2007-03-31,2000-05-31,6031,2874.38,12554.90,3.90,81,0,0,2874.36,2.00
65,1469639,2000-02-01,2007-03-31,2000-07-31,6031,3632.28,15971.90,5.90,79,0,0,3632.30,1.00
66,1469639,2000-02-01,2007-03-31,2000-08-31,6031,2102.50,18074.40,7.00,78,0,0,2102.50,1.10
67,1469639,2000-02-01,2007-03-31,2000-09-30,6031,732.88,18807.20,8.00,77,0,0,732.80,1.00
161,1469642,2000-02-16,2007-03-31,2001-12-31,6031,74568.00,83536.50,22.50,62,0,0,74568.04,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
382945,2016607,2000-12-29,2024-05-31,2001-09-30,6031,-150.58,819366.00,9.00,272,0,0,819366.00,0.60
383076,2016635,2000-12-29,2008-01-31,2001-02-28,6031,1065.87,17675.70,2.00,83,0,0,1065.80,0.90
383077,2016635,2000-12-29,2008-01-31,2001-03-31,6031,4627.38,22303.10,3.00,82,0,0,4627.40,1.00
383078,2016635,2000-12-29,2008-01-31,2001-04-30,6031,55.84,22358.90,4.00,81,0,0,55.80,1.00


In [255]:
df

TypeError: first() missing 1 required positional argument: 'offset'

In [249]:
df[(df['delta']==0)]

,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt,MnthsFrom_Origination,MnthsFrom_Maturity,PaidOff,PrepayAmt,delta,double_entry
17,1469636,2000-01-01,2005-01-31,2003-01-28,1510,15995.30,15995.30,36.90,24,0,0,0.00,19.90
973,1469670,2000-07-13,2007-07-31,2001-11-30,6031,1064.19,49830.60,16.60,67,0,0,0.00,1.00
1384,1480771,2000-01-03,2009-04-29,2006-11-20,1510,-80999.00,0.00,82.60,29,1,-80999.00,0.00,15.40
1456,1480781,2000-01-03,2007-11-15,2017-09-28,1510,-195994.00,0.00,212.80,-118,1,-195994.00,0.00,127.10
1720,1480799,2000-01-03,2010-01-31,2004-05-19,1510,158631.00,158631.00,52.50,68,0,0,0.00,2.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381444,2016467,2000-12-28,2026-11-03,2008-07-31,1510,2394.00,490143.00,91.10,219,0,0,0.00,1.30
381487,2016475,2000-12-28,2008-02-28,2006-05-17,1510,-42096.40,0.00,64.60,21,1,-42096.40,0.00,39.20
381811,2016521,2000-12-29,2016-01-26,2003-09-09,1510,511671.00,511671.00,32.30,148,0,0,0.00,12.30
382099,2016529,2000-12-29,2011-04-10,2009-06-30,6031,61587.30,246349.00,102.00,21,0,0,0.00,2.00


In [257]:
# show_slice(df[df['double_entry']>5],'1481527')
df[df['double_entry']>5]

,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt,MnthsFrom_Origination,MnthsFrom_Maturity,PaidOff,PrepayAmt,delta,double_entry
800,1469665,2000-08-16,2020-10-31,2005-08-10,6031,-108803.00,0.00,59.80,182,1,-108803.00,-108803.00,50.30
1015,1476786,2000-01-20,2002-08-03,2002-08-29,6031,-220691.00,0.00,31.30,0,1,-220691.00,-222928.00,11.00
1734,1480806,2000-01-03,2018-01-31,2001-07-31,6031,-49393.70,0.00,18.90,198,1,-49393.70,-49393.70,6.00
2040,1480938,2000-01-04,2020-07-31,2012-03-31,6031,-372797.00,0.00,146.90,100,1,-372797.00,-372797.00,85.10
2481,1480984,2000-01-04,2008-02-04,2004-02-29,6031,-233477.00,0.00,49.80,47,1,-233477.00,-233477.00,20.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381331,2016458,2000-12-28,2020-09-30,2004-05-31,6031,-518896.00,0.00,41.10,196,1,-518896.00,-518896.00,6.00
381392,2016466,2000-12-28,2011-02-28,2012-04-30,6031,-575014.00,0.00,136.10,-14,1,-575014.00,-575014.00,72.10
381672,2016497,2000-12-28,2008-03-31,2009-03-31,6031,-3653.15,0.00,99.10,-11,1,-3653.15,-3653.15,19.00
382837,2016579,2000-12-29,2021-07-31,2014-08-31,6031,-91797.00,0.00,164.00,82,1,-91797.00,-91797.00,30.00


In [159]:
# get the minimum number of double entries for each observation nmb
target = df[['ObservationNmb','double_entry']].groupby('ObservationNmb').min()
# get a list of the loans with double entries
target = target[target['double_entry']==0].index.to_list()
# rewindow to target
target = df.loc[df['ObservationNmb'].isin(target)]
# slice to where double entry occurs for those loans
target = target[target['double_entry']==0]


,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt,MnthsFrom_Origination,MnthsFrom_Maturity,PaidOff,PrepayAmt,double_entry,delta
18,1469636,2000-01-01,2005-01-31,2003-01-28,6031,-15995.30,0.00,36.90,24,1,-15995.30,0.00,-15995.30
511,1469652,2000-06-30,2020-06-30,2009-06-30,6031,-52763.30,52172.10,108.00,132,0,0,0.00,-52171.90
641,1469653,2000-05-09,2005-06-30,2001-06-30,6031,12000.00,90500.00,13.70,48,0,0,0.00,90500.00
973,1469670,2000-07-13,2007-07-31,2001-11-30,6031,-1064.19,48766.40,16.60,67,0,0,0.00,-1064.20
1219,1480769,2000-01-03,2020-01-31,2001-11-30,6031,75179.50,300718.00,22.90,218,0,0,0.00,598.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381436,2016548,2000-12-29,2026-03-31,2009-06-30,6031,-45006.50,133302.00,102.00,201,0,0,0.00,-44434.00
381753,2016573,2000-12-29,2021-08-31,2009-06-30,6031,-178687.00,526593.00,102.00,146,0,0,0.00,-175531.00
381914,2016576,2000-12-29,2013-01-31,2009-06-30,6031,-7413.09,16662.10,102.00,43,0,0,0.00,-5554.10
382021,2016579,2000-12-29,2021-07-31,2009-06-30,6031,-49518.50,146008.00,102.00,145,0,0,0.00,-48670.00


delta
ObservationNmb EffectiveDt GeneralLedgerCd TransactionBalanceAmt       
1483145        2000-10-31  6031            155170.00                  2
1484682        2001-12-07  1510            65739.60                   2
1485370        2004-12-08  1510            289838.00                  2
1488421        2009-05-29  1510            153141.00                  2
1491957        2008-08-28  1510            398643.00                  2
               2009-08-24  1510            397143.00                  2
1497116        2002-03-08  1510            137533.00                  2
1497727        2016-02-01  1510            0.00                       2
1497854        2010-08-17  1510            51879.90                   2
1499375        2003-08-14  1510            58292.30                   2
1499485        2004-08-16  1510            17057.70                   2
1501321        2002-12-11  1510            1263630.00                 2
1501920        2006-09-08  1510            105218.00                  2
1503841        2004-08-30  1510            55890.60                   2
1504283        2016-04-06  1510            0.00                       2
1504491        2003-06-26  1510            380674.00                  2
1505241        2015-11-02  1510            294396.00                  2
               2018-01-24  1510            70620.50                   2
1507589        2002-02-28  6031            159524.00                  2
1507593        2002-02-28  6031            55833.20                   2
1509668        2006-02-27  1510            440070.00                  2
1510790        2003-02-28  6031            897503.00                  2
1511174        2004-02-29  6031            38286.60                   2
1516262        2005-08-31  6031            0.00                       2
2007104        2001-05-31  6031            334658.00                  2
2009614        2009-05-18  1510            354794.00                  2
2009815        2001-02-28  6031            49341.00                   2
2012314        2005-01-25  1510            187819.00                  2
2013540        2005-08-04  1510            80725.30                   2
2013851        2012-06-05  1510            327194.00                  2
2015478        2007-07-02  1510            626963.00                  2
2015734        2008-05-08  1510            0.00                       2
                                           587764.00                  3
2015916        2005-06-29  1510            84399.50                   2
2016177        2009-06-01  1510            222067.00                  2

In [110]:
df.loc[df.index==256575]

,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt,MnthsFrom_Origination,MnthsFrom_Maturity,PaidOff,PrepayAmt,double_entry
256575,1511228,2000-08-18,2012-08-31,2005-11-01,1510,9467.64,893377.00,62.50,81,0,0,16.10


In [172]:
df.loc[df.index==230315]

,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt,MnthsFrom_Origination,MnthsFrom_Maturity,PaidOff,PrepayAmt,double_entry,delta
230315,1507589,2000-07-20,2007-09-30,2002-01-31,6031,-2380.96,161905.00,18.40,67,0,0,1.00,-2381.00


In [167]:
show_slice(df[df['double_entry']==0],'1507589')

,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt,MnthsFrom_Origination,MnthsFrom_Maturity,PaidOff,PrepayAmt,double_entry,delta
230317,1507589,2000-07-20,2007-09-30,2002-02-28,6031,-2380.96,159524.00,19.30,67,0,0,0.00,0.00
230318,1507589,2000-07-20,2007-09-30,2002-02-28,6031,0.01,159524.00,19.30,67,0,0,0.00,0.00


In [200]:
df[(df['TransactionAmt']<=1)&(df['TransactionAmt']>=-1)&(df['MnthsFrom_Maturity']<0)]
df[df['MnthsFrom_Maturity']<0]

,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt,MnthsFrom_Origination,MnthsFrom_Maturity,PaidOff,PrepayAmt,double_entry,delta
1454,1480781,2000-01-03,2007-11-15,2017-09-28,1510,-195994.00,0.00,212.80,-118,1,-195994.00,127.10,0.00
2194,1480962,2000-01-04,2005-05-31,2005-07-31,6031,-660.68,14383.30,66.90,-2,0,0,1.10,-660.70
2195,1480962,2000-01-04,2005-05-31,2005-08-31,6031,-671.64,13711.60,67.90,-3,0,0,1.00,-671.70
2196,1480962,2000-01-04,2005-05-31,2005-09-30,6031,-682.28,13029.40,68.90,-4,0,0,1.00,-682.20
2197,1480962,2000-01-04,2005-05-31,2005-10-31,6031,-756.85,12272.50,69.90,-5,0,0,1.00,-756.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380887,2016497,2000-12-28,2008-03-31,2009-03-31,6031,-3653.15,0.00,99.10,-11,1,-3653.15,19.00,-3653.15
381010,2016519,2000-12-28,2011-02-28,2011-07-31,6031,-56331.60,0.00,127.00,-5,1,-56331.60,5.00,-56331.60
381028,2016521,2000-12-29,2016-01-26,2016-06-08,1510,9650.00,521321.00,185.30,-4,0,0,153.00,521321.00
381029,2016521,2000-12-29,2016-01-26,2016-06-13,1510,-521321.00,0.00,185.50,-4,1,-521321.00,0.20,-521321.00


In [197]:
show_slice(df, '2011791')

,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt,MnthsFrom_Origination,MnthsFrom_Maturity,PaidOff,PrepayAmt,double_entry,delta
342154,2011791,2000-11-16,2015-12-31,2000-12-31,6031,730000.00,730000.00,1.50,179,0,0,NaN,NaN
342155,2011791,2000-11-16,2015-12-31,2001-03-31,6031,-2019.01,727981.00,4.40,177,0,0,2.90,-2019.00
342156,2011791,2000-11-16,2015-12-31,2001-04-30,6031,-1305.01,726676.00,5.40,176,0,0,1.00,-1305.00
342157,2011791,2000-11-16,2015-12-31,2001-05-31,6031,-383.21,726293.00,6.40,175,0,0,1.00,-383.00
342158,2011791,2000-11-16,2015-12-31,2001-06-30,6031,-3168.20,723125.00,7.40,174,0,0,1.00,-3168.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
342261,2011791,2000-11-16,2015-12-31,2015-08-31,6031,-9259.65,29206.20,177.40,4,0,0,1.00,-9259.60
342262,2011791,2000-11-16,2015-12-31,2015-09-30,6031,-9317.97,19888.20,178.40,3,0,0,1.00,-9318.00
342263,2011791,2000-11-16,2015-12-31,2015-11-30,6031,-19804.50,83.69,180.40,1,1,-19804.50,2.00,-19804.51
342264,2011791,2000-11-16,2015-12-31,2016-01-31,6031,-82.70,0.99,182.50,-1,1,-82.70,2.10,-82.70


In [286]:
x= test.groupby('ObservationNmb')[['delta']].sum().astype(float).reset_index()
x = x[x['delta']!=0]
x = x[x['delta'].abs() > 0.1]
x[x['delta']>0]

,ObservationNmb,delta
11,1469653,90500.00
89,1481237,200887.00
240,1482064,279274.00
268,1482186,41602.00
281,1482237,209365.00
...,...,...
6955,2015491,895513.00
7008,2015919,25212.90
7022,2016070,15078.00
7061,2016322,100647.00


In [291]:

show_slice(test[test['mnth_diff']<=1], '2015491')

,ObservationNmb,EffectiveDt,MnthsFrom_Origination,TransactionAmt,TransactionBalanceAmt,delta,mnth_diff
357800,2015491,2001-01-31,1,1084880.00,1084880.00,1084880.00,-8.00
357801,2015491,2001-02-28,2,-15956.70,1068930.00,-15950.00,1.00
357802,2015491,2001-03-31,3,-15818.40,1053110.00,-15820.00,1.00
357803,2015491,2001-04-30,4,-15555.00,1037550.00,-15560.00,1.00
357805,2015491,2001-07-31,7,-36181.40,985784.00,-36186.00,1.00
357806,2015491,2001-08-31,8,-16371.20,969413.00,-16371.00,1.00
357808,2015491,2001-11-30,11,-18390.60,933156.00,-18390.00,1.00
357809,2015491,2001-12-31,12,-19206.40,913949.00,-19207.00,1.00
357810,2015491,2002-01-31,13,-895513.00,0.00,-913949.00,1.00
357811,2015491,2002-01-31,13,-18436.60,895513.00,895513.00,0.00


In [290]:
show_slice(df, '2016322')

,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt,MnthsFrom_Origination,MnthsFrom_Maturity,PaidOff,PrepayAmt
363327,2016322,2000-12-27,2027-01-31,2001-01-31,6031,304500.00,304500.00,1,312,0,0
363328,2016322,2000-12-27,2027-01-31,2001-03-31,6031,-899.37,303601.00,3,310,0,0
363329,2016322,2000-12-27,2027-01-31,2001-04-30,6031,-165.04,303436.00,4,309,0,0
363330,2016322,2000-12-27,2027-01-31,2001-05-31,6031,-36.00,303400.00,5,308,0,0
363331,2016322,2000-12-27,2027-01-31,2001-06-30,6031,-260.74,303139.00,6,307,0,0
...,...,...,...,...,...,...,...,...,...,...,...
363492,2016322,2000-12-27,2027-01-31,2021-04-30,6031,-1654.29,107343.00,244,69,0,0
363493,2016322,2000-12-27,2027-01-31,2021-05-31,6031,-1674.95,105668.00,245,68,0,0
363494,2016322,2000-12-27,2027-01-31,2021-06-30,6031,-1667.72,104001.00,246,67,0,0
363495,2016322,2000-12-27,2027-01-31,2021-07-31,6031,-1639.84,102361.00,247,66,0,0


In [135]:
chrt = df[['ObservationNmb','EffectiveDt', 'TransactionBalanceAmt','PrepayAmt']]
chrt['Year'] = chrt['EffectiveDt'].dt.year
chrt['Month'] = chrt['EffectiveDt'].dt.month
temp = chrt.groupby(['ObservationNmb', 'Year', 'Month'])[['TransactionBalanceAmt']].first().reset_index()
temp = chrt.groupby(['Year', 'Month'])[['TransactionBalanceAmt']].sum().reset_index()
temp2 = chrt.groupby(['Year', 'Month'])[['PrepayAmt']].sum().reset_index()
temp3 = pd.concat([temp,temp2[['PrepayAmt']]], axis=1)

In [ ]:
show_slice(df, '2016284')

In [176]:
for df_c in df_list:
    try:
        v =(df_c['ObservationNmb'].mode()[0])
        if v == '2016284':
            display(df_c)
    except KeyError:
        continue

,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt
0,2016284,2000-12-27,2009-08-31,2001-01-31,6031,370600.00,370600.00
1,2016284,2000-12-27,2009-08-31,2001-01-31,6031,370600.00,370600.00
2,2016284,2000-12-27,2009-08-31,2001-03-31,6031,-2518.21,368082.00
3,2016284,2000-12-27,2009-08-31,2001-03-31,6031,-2518.21,368082.00
4,2016284,2000-12-27,2009-08-31,2001-04-30,6031,-2153.71,365928.00
...,...,...,...,...,...,...,...
97,2016284,2000-12-27,2009-08-31,2006-12-31,6031,-3613.49,185708.00
98,2016284,2000-12-27,2009-08-31,2007-07-12,1510,185708.00,185708.00
99,2016284,2000-12-27,2009-08-31,2007-07-12,6031,-185708.00,0.00
100,2016284,2000-12-27,2009-08-31,2009-12-08,1510,22398.90,208107.00


In [175]:
df

,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt


In [151]:
# # Buil notional mapping from observation number to notional
# not_map = build_not_map(ls)
# # Build a column that calculates the percentage of principal a transaction has paid off
# df['pct_outstanding'] = df['TransactionAmt']/(df['ObservationNmb'].apply(lambda x: not_map[x]))


,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt,MnthsFrom_Origination,MnthsFrom_Maturity,PaidOff,PrepayAmt,pct_outstanding,VoluntaryPrepay
0,1469636,2000-01-01,2005-01-31,2000-01-31,6031,20000.00,20000.00,0,60,0,0,1.00,0
0,1469638,2000-01-31,2005-05-31,2000-05-31,6031,120000.00,120000.00,4,60,0,0,1.00,0
0,1469639,2000-02-01,2007-03-31,2000-03-31,6031,9680.54,9680.54,1,84,0,0,0.48,0
2,1469639,2000-02-01,2007-03-31,2000-05-31,6031,2874.38,12554.90,3,82,0,0,0.14,0
6,1469639,2000-02-01,2007-03-31,2000-07-31,6031,3632.28,15971.90,5,80,0,0,0.18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2016629,2000-12-29,2021-01-31,2006-12-31,6031,67673.30,298954.00,72,169,0,0,0.18,0
0,2016631,2000-12-29,2016-02-28,2001-02-28,6031,99643.70,99643.70,2,180,0,0,1.00,0
0,2016635,2000-12-29,2008-01-31,2001-01-31,6031,16609.90,16609.90,1,84,0,0,0.66,0
4,2016635,2000-12-29,2008-01-31,2001-03-31,6031,4627.38,22303.10,3,82,0,0,0.19,0


In [167]:

# Build Voluntary Prepayment Column
# df.to_csv('2000_principal_pmt_history.csv')
# df[(df['EffectiveDt'].dt.year==2008)&(df['EffectiveDt'].dt.month==3)]


In [74]:
test1= df[['ObservationNmb','EffectiveDt','TransactionBalanceAmt']]
test1['Year'] = test1['EffectiveDt'].dt.year
test1['Month'] = test1['EffectiveDt'].dt.month
test1 = test1[['ObservationNmb','Year','Month','TransactionBalanceAmt']]
# test1.groupby([ test1['EffectiveDt'].dt.year, test1['EffectiveDt'].dt.month ]).sum().to_csv('2000_total_amts.csv')
test2 = test1.groupby([test1['ObservationNmb'], test1['Year'], test1['Month'] ]).first().reset_index()
test2[['TransactionBalanceAmt']].groupby([test1['Year'], test1['Month'] ]).sum().to_csv('csvs/cleaning_test.csv')

In [79]:
df[(df['EffectiveDt'].dt.year==2009)&(df['EffectiveDt'].dt.month==6)].sort_values('TransactionBalanceAmt', ascending=False)

,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt,MnthsFrom_Origination,MnthsFrom_Maturity,PaidOff,PrepayAmt
116802,1493742,2000-04-10,2025-04-25,2009-06-30,6031,1158810.00,1622310.00,110,190,0,0
349003,2009729,2000-10-31,2025-09-30,2009-06-30,6031,842675.00,1474680.00,104,195,0,0
69950,1488818,2000-03-08,2025-09-30,2009-06-30,6031,587042.00,1264390.00,111,195,0,0
285907,1513460,2000-09-06,2020-10-31,2009-06-30,6031,489638.00,1101670.00,105,136,0,0
380346,2013463,2000-12-07,2015-12-31,2009-06-02,1510,-8999.06,1067850.00,102,78,0,0
...,...,...,...,...,...,...,...,...,...,...,...
299933,1515034,2000-09-15,2010-09-30,2009-06-10,6031,-516233.00,0.00,105,15,1,-516233.00
208505,1503883,2000-06-22,2006-08-01,2009-06-05,6031,-351625.00,0.00,108,-34,1,-351625.00
83423,1490118,2000-03-16,2015-03-31,2009-06-17,6031,-426117.00,0.00,111,69,1,-426117.00
397960,2015414,2000-12-19,2011-02-28,2009-06-17,6031,-41991.40,0.00,102,20,1,-41991.40


In [82]:
test = df[df['ObservationNmb']== '1493742']
test[(test['EffectiveDt'].dt.year==2009)]

,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt,MnthsFrom_Origination,MnthsFrom_Maturity,PaidOff,PrepayAmt
116797,1493742,2000-04-10,2025-04-25,2009-01-31,6031,-13436.10,1692620.00,105,195,0,0
116798,1493742,2000-04-10,2025-04-25,2009-03-31,6031,-28416.60,1664210.00,107,193,0,0
116799,1493742,2000-04-10,2025-04-25,2009-04-30,6031,-13980.50,1650230.00,108,192,0,0
116800,1493742,2000-04-10,2025-04-25,2009-05-31,6031,-14251.60,1635980.00,109,191,0,0
116801,1493742,2000-04-10,2025-04-25,2009-06-30,6031,-1172480.00,463493.00,110,190,0,0
116802,1493742,2000-04-10,2025-04-25,2009-06-30,6031,1158810.00,1622310.00,110,190,0,0
116803,1493742,2000-04-10,2025-04-25,2009-07-31,6031,-14790.40,1607520.00,111,189,0,0
116804,1493742,2000-04-10,2025-04-25,2009-08-31,6031,-14216.70,1593300.00,112,188,0,0
116805,1493742,2000-04-10,2025-04-25,2009-09-30,6031,-14688.70,1578610.00,113,187,0,0
116806,1493742,2000-04-10,2025-04-25,2009-10-31,6031,-14118.90,1564490.00,114,186,0,0


In [230]:
(df1[(df1['TransactionBalanceAmt'].values==df['TransactionAmt'].values) & (df1['TransactionAmt']<0) ])

,Unnamed: 0,ObservationNmb,Origination_dt,MaturityDt,EffectiveDt,GeneralLedgerCd,TransactionAmt,TransactionBalanceAmt,MnthsFrom_Origination,MnthsFrom_Maturity,PaidOff,pct_outstanding,VoluntaryPrepay
1885,0,1480845,1/3/2000,2/28/2025,2000-02-29,6031,-1768000.00,-1768000.00,1,300,1,-1.00,0.00
2423,0,1480975,1/4/2000,1/26/2007,2000-02-29,6031,-93750.00,-93750.00,1,83,1,-0.84,0.00
3064,0,1481027,1/4/2000,2/28/2010,2000-02-29,6031,-349835.00,-349835.00,1,120,1,-1.00,0.00
3385,0,1481143,1/5/2000,2/29/2020,2000-02-29,6031,-469966.00,-469966.00,1,240,1,-1.00,0.00
3965,0,1481212,1/5/2000,2/28/2010,2000-02-29,6031,-134300.00,-134300.00,1,120,1,-0.99,0.00
4058,0,1481219,1/5/2000,2/28/2003,2000-02-29,6031,-12739.20,-12739.20,1,36,1,-0.98,0.00
4979,0,1481313,1/5/2000,5/29/2006,2000-02-29,6031,-69206.20,-69206.20,1,75,1,-0.16,0.00
5051,0,1481321,1/6/2000,2/28/2001,2000-02-29,6031,-130229.00,-130229.00,1,12,1,-0.52,0.00
5778,0,1481372,1/6/2000,2/28/2015,2000-02-29,6031,-259401.00,-259401.00,1,180,1,-1.00,0.00
5933,0,1481381,1/6/2000,2/29/2020,2000-02-29,6031,-246677.00,-246677.00,1,240,1,-0.99,0.00


In [ ]:
test1 = test1[test1['PrepayAmt']<0]
test1.groupby([ test1['EffectiveDt'].dt.year, test1['EffectiveDt'].dt.month ]).mean()

In [72]:
test2= test[['EffectiveDt','TransactionBalanceAmt','PrepayAmt', 'PaidOff']]
test2.groupby([ test2['EffectiveDt'].dt.year, test2['EffectiveDt'].dt.month ]).sum().to_csv('2000_total_counts.csv')

In [73]:
test.to_csv('2000_principal_pmt_history.csv')